## dbt Cloud API Example for Updating Users
- As a note, I would recommend doing these command vis the [dbt Cloud Terraform Provider](https://registry.terraform.io/providers/dbt-labs/dbtcloud/latest) instead of scripting with the APIs but provided scripting examples below

In [2]:
import requests
from datetime import datetime
import json
import pprint

In [ ]:
dbt_cloud_api_token = "dbtc_1234587237823782378237823" # Update to your token
dbt_cloud_account_id = 123456 # Update to account id
# Set up the headers for authentication
api_headers = {
    'Authorization': f'Token {dbt_cloud_api_token}',
    'Content-Type': 'application/json'
}

___

### Example 1: Removing a user from a dbt Cloud Group

In [ ]:
dbt_cloud_user_id = 21833 # Update to a given users ID

#### First i'll list the groups that the user is in

In [35]:
# Define the URL to get the groups that the user is in
url = f'https://cloud.getdbt.com/api/v2/accounts/{dbt_cloud_account_id}/users/{dbt_cloud_user_id}'

# Make the GET request to fetch the groups associated with the user
response = requests.get(url, headers=api_headers)

# Check the response status
if response.status_code == 200:
    dbt_cloud_user_info = response.json()
else:
    print(f'Failed to fetch groups for user {dbt_cloud_user_id}. Status code: {response.status_code}')

# Extract the groups from the "permissions" section
groups = []
for permission in dbt_cloud_user_info['data'].get('permissions', []):
    if permission.get('account_id') == dbt_cloud_account_id:
        groups.extend(permission.get('groups', []))
# Print out the group IDs and names
if groups:
    print("Groups that the user is in within the dbt Cloud Account:")
    for group in groups:
        print(f"ID: {group['id']}, Name: {group['name']}")
else:
    print("No groups found for the user.")

Groups that the user is in within the dbt Cloud Account:
ID: 782548, Name: Developers
ID: 782545, Name: Account Admins


#### Second i'll update the users groups
- You'll notice in this API call i'm removing this user from group `782545` which is named `Account Admins` 
- I remove this user by assigning them only to the groups that I want them to be in, which in this case is group `782548` which is named `Developers`
  - By only assigning them to the groups that I want them to be in, it will remove them from all other groups

In [36]:
desired_user_groups = [782548] 

# Define the URL for assigning groups
url = f'https://cloud.getdbt.com/api/v3/accounts/{dbt_cloud_account_id}/assign-groups/'

# Create the payload
payload = {
    "user_id": dbt_cloud_user_id,
    "desired_group_ids": desired_user_groups
}

# Make the POST request to assign the user to the specified groups
response = requests.post(url, headers=api_headers, json=payload)

# Check the response status
if response.status_code == 200:
    print(f'Successfully updated user {dbt_cloud_user_id} groups')
else:
    print(f'Failed to update user {dbt_cloud_user_id} groups. Status code: {response.status_code}')
    print('Response:', response.json())

Successfully updated user 72232 groups


#### Third i'll check the groups that the user is in to confirm they were removed from any groups I do not want them in

In [37]:
# Define the URL to get the groups that the user is in
url = f'https://cloud.getdbt.com/api/v2/accounts/{dbt_cloud_account_id}/users/{dbt_cloud_user_id}'

# Make the GET request to fetch the groups associated with the user
response = requests.get(url, headers=api_headers)

# Check the response status
if response.status_code == 200:
    dbt_cloud_user_info = response.json()
else:
    print(f'Failed to fetch groups for user {dbt_cloud_user_id}. Status code: {response.status_code}')

# Extract the groups from the "permissions" section
groups = []
for permission in dbt_cloud_user_info['data'].get('permissions', []):
    if permission.get('account_id') == dbt_cloud_account_id:
        groups.extend(permission.get('groups', []))
# Print out the group IDs and names
if groups:
    print("Groups that the user is in within the dbt Cloud Account:")
    for group in groups:
        print(f"ID: {group['id']}, Name: {group['name']}")
else:
    print("No groups found for the user.")

Groups that the user is in within the dbt Cloud Account:
ID: 782548, Name: Developers


### Example 2: Changing the License type for a particular user (Eg: from Read-only to Developer and vice-versa)

#### First i'll put back the user's current license type

In [41]:
# Define the URL to get the license type that the user has
url = f'https://cloud.getdbt.com/api/v2/accounts/{dbt_cloud_account_id}/users/{dbt_cloud_user_id}'

# Make the GET request to fetch the groups associated with the user
response = requests.get(url, headers=api_headers)

# Check the response status
if response.status_code == 200:
    dbt_cloud_user_info = response.json()
else:
    print(f'Failed to fetch groups for user {dbt_cloud_user_id}. Status code: {response.status_code}')

# Find the license type for the specified account ID
license_type = None
for permission in dbt_cloud_user_info['data'].get('permissions', []):
    if permission.get('account_id') == dbt_cloud_account_id:
        license_type = permission.get('license_type')
        license_assignment_id = permission.get('id')
        break

# Output the result
if license_type:
    print(f"The license type for user {dbt_cloud_user_id} in the account {dbt_cloud_account_id} is: {license_type}, the id for this license assignment is {license_assignment_id}")
else:
    print(f"No license type for user {dbt_cloud_user_id} found in the account {dbt_cloud_account_id}.")

The license type for user 72232 in the account 164789 is: read_only, the id for this license assignment is 341814


#### Second i'll update the users license type from developer to read-only
- This can be done in reverse order as well (i.e. from read-only to developer)
- Note that in order to make this call I need the `license_assignment_id` which is provided from the above API call

In [43]:
# This can be programmatically pull from the above response but hard coding it to help with writing out the example
license_assignment_id = 341814

# need to be either 'read-only' or 'developer'
desired_license_type = 'developer'

url = f'https://cloud.getdbt.com/api/v2/accounts/{dbt_cloud_account_id}/permissions/{license_assignment_id}/'

# Create the payload
payload = {
    "id": license_assignment_id,
    "account_id": dbt_cloud_account_id,
    "user_id": dbt_cloud_user_id,
    "license_type": desired_license_type
}

# Make the PUT request to update the permission
response = requests.post(url, headers=api_headers, json=payload)

# Check the response status
if response.status_code == 200:
    print(f'Successfully updated user {dbt_cloud_user_id} to license type {desired_license_type}')
else:
    print(f'Failed to update user license type. Status code: {response.status_code}')
    print('Response:', response.json())

Successfully updated user 72232 to license type developer


#### Third i'll check to make sure the user's license type was successfully updated    

In [44]:
# Define the URL to get the license type that the user has
url = f'https://cloud.getdbt.com/api/v2/accounts/{dbt_cloud_account_id}/users/{dbt_cloud_user_id}'

# Make the GET request to fetch the groups associated with the user
response = requests.get(url, headers=api_headers)

# Check the response status
if response.status_code == 200:
    dbt_cloud_user_info = response.json()
else:
    print(f'Failed to fetch groups for user {dbt_cloud_user_id}. Status code: {response.status_code}')

# Find the license type for the specified account ID
license_type = None
for permission in dbt_cloud_user_info['data'].get('permissions', []):
    if permission.get('account_id') == dbt_cloud_account_id:
        license_type = permission.get('license_type')
        license_assignment_id = permission.get('id')
        break

# Output the result
if license_type:
    print(f"The license type for user {dbt_cloud_user_id} in the account {dbt_cloud_account_id} is: {license_type}, the id for this license assignment is {license_assignment_id}")
else:
    print(f"No license type for user {dbt_cloud_user_id} found in the account {dbt_cloud_account_id}.")

The license type for user 72232 in the account 164789 is: developer, the id for this license assignment is 341814


### Example 3: Removing a user from the dbt Cloud account all together

#### First i'll list all users on the account

In [5]:
# Define the URL to get the groups that the user is in
url = f'https://cloud.getdbt.com/api/v2/accounts/{dbt_cloud_account_id}/users/'

# Make the GET request to fetch the groups associated with the user
response = requests.get(url, headers=api_headers)

# Check the response status
if response.status_code == 200:

    dbt_cloud_user_info = response.json()['data']

    all_users_list = []
    
    # this is optional but nice to slim down the response 
    for user in dbt_cloud_user_info:
        
        user_info = {
            "name": f"{user['first_name']} {user['last_name']}",
            "email": user["email"],
            "id": user["id"],
            # this is how we remove someone from dbt Cloud
            "user_permission_id": user["permissions"][0]["id"]
        }

        #  append to the list    
        all_users_list.append(user_info)

    # log the users, pretty printing this but this is optional
    pprint.pprint(all_users_list, compact=True)
else:
    print(f'Failed to fetch all users for account {dbt_cloud_account_id}. Status code: {response.status_code}')

[{'email': 'steve.dowling@fishtownanalytics.com',
  'id': 94614,
  'name': 'Steve Dowling',
  'user_permission_id': 218348},
 {'email': 'steve.dowling@dbtlabs.com',
  'id': 143722,
  'name': 'steve dowling',
  'user_permission_id': 225646}]


#### Second i'll remove the user i'm looking to remove from the account

In [6]:
# grabbed from listing users above
user_email = 'steve.dowling@dbtlabs.com'
user_id = 143722
user_permission_id = 225646

# Construct the URL
delete_user_api_endpoint = f"https://cloud.getdbt.com/api/v2/accounts/{dbt_cloud_account_id}/permissions/{user_permission_id}/"

# payload to delete user
delete_user_payload = {
        "account_id": dbt_cloud_account_id,
        "id": user_permission_id,
        "user_id": user_id,
        "state": 2
    }


# Make the GET request
response = requests.post(delete_user_api_endpoint, headers=api_headers, json=delete_user_payload)

# Check the response
if response.status_code == 200:
    print(f"Successfully removed user {user_email} with permission_id {user_permission_id} from dbt Cloud Account {dbt_cloud_account_id}")
else:
    print(f"Failed to fetch data. Status code: {response.status_code} {response.text}")

Successfully removed user steve.dowling@dbtlabs.com with permission_id 225646 from dbt Cloud Account 164789


#### Third i'll list all users on the account again to confirm user have been removed

In [7]:
# Define the URL to get the groups that the user is in
url = f'https://cloud.getdbt.com/api/v2/accounts/{dbt_cloud_account_id}/users/'

# Make the GET request to fetch the groups associated with the user
response = requests.get(url, headers=api_headers)

# Check the response status
if response.status_code == 200:

    dbt_cloud_user_info = response.json()['data']

    all_users_list = []
    
    # this is optional but nice to slim down the response 
    for user in dbt_cloud_user_info:
        
        user_info = {
            "name": f"{user['first_name']} {user['last_name']}",
            "email": user["email"],
            "id": user["id"],
            # this is how we remove someone from dbt Cloud
            "user_permission_id": user["permissions"][0]["id"]
        }

        #  append to the list    
        all_users_list.append(user_info)

    # log the users, pretty printing this but this is optional
    pprint.pprint(all_users_list, compact=True)
else:
    print(f'Failed to fetch all users for account {dbt_cloud_account_id}. Status code: {response.status_code}')

[{'email': 'steve.dowling@fishtownanalytics.com',
  'id': 94614,
  'name': 'Steve Dowling',
  'user_permission_id': 218348}]


___
END OF SCRIPT
___